In [1]:
# **************************
# Author: Sebastien Vezina
# Date: 05-Nov-2020
# **************************
import os
import requests
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from functools import partial

from alpha_vantage.foreignexchange import ForeignExchange

# Install package before importing: pip install alpha-vantage
# https://pypi.org/project/alpha-vantage/

In [2]:
# Load .env enviroment variables
load_dotenv()
alpha_vantage_key = os.getenv("ALPHAVANTAGE")

In [3]:
#Pull CAD Data
fx = ForeignExchange(key=alpha_vantage_key,output_format='pandas')
cad_data, meta_data = fx.get_currency_exchange_daily(from_symbol='CAD', to_symbol='USD', outputsize='full')

cad_data.head()

,1. open,2. high,3. low,4. close
date,,,,
2014-11-24,0.8893,0.8907,0.8835,0.8859
2014-11-25,0.8856,0.8901,0.8832,0.8880
2014-11-26,0.8881,0.8904,0.8847,0.8887
2014-11-27,0.8883,0.8899,0.8802,0.8816
2014-11-28,0.8812,0.8820,0.8735,0.8755


In [4]:
# Save to CSV file
cad_data.to_csv('../../02-Data/Dirty Data/Market Data/CAD_5yr_drt.csv')